In [3]:
%%writefile mapper.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, word_count = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    word_count = int(word_count)
    print >> sys.stderr, "reporter:counter:Wiki stats,Word count,%d" % word_count
    print "%d\t%s" % (word_count, word)

Writing mapper.py


In [8]:
%%writefile reducer.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

current_key = None
word_sum = 0

Overwriting reducer.py


In [1]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer.py


In [11]:
%%writefile T1mapper_out.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

CountDict = {} # key is count, value is list of all words that have that count

for line in sys.stdin:
    try:
        word, word_count = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    word_count = int(word_count)
    if word_count in CountDict:
        CountDict[word_count].append(word)
    else:
        CountDict[word_count] = [word]

word_sep = ';'
for word_count in CountDict:
    print "%d\t%s" % (word_count, word_sep.join(CountDict[word_count]))

Writing T1mapper_out.py


In [12]:
%%writefile T1combiner.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

CountDict = {} # key is count, value is list of all words that have that count

for line in sys.stdin:
    try:
        word_count, word = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    word_count = int(word_count)
    if word_count in CountDict:
        CountDict[word_count].append(word)
    else:
        CountDict[word_count] = [word]

word_sep = ';'
for word_count in CountDict:
    print "%d\t%s" % (word_count, word_sep.join(CountDict[word_count]))

Overwriting T1combiner.py


In [13]:
%%writefile T1reducer.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

# Adds line of word_count and associated words to the WordCountList
# in the correct index such that the list retains reverse sort order in word_count
def AddElementtoList (WordCountList, word_count, words, curr_index):

    if len(WordCountList) == 0: # Insert the first element in the list
        test_index = 0
        WordCountList.append([word_count, words])
        return WordCountList, test_index
        
    if word_count > WordCountList[curr_index][0]:
        test_index = 0
    else:
        test_index = curr_index
        
    while WordCountList[test_index][0] > word_count:
        if test_index < len(WordCountList):
            test_index += 1
        if test_index == len(WordCountList):
            break
    # Now, consider the cases for where to insert this data into the list
    if test_index == len(WordCountList): # Append this data to the list's end
        WordCountList.append ([word_count, words])

    elif WordCountList[test_index][0] == word_count: # Words of this length already exist in the list
        WordCountList[test_index][1] = WordCountList[test_index][1] + words
        
    elif test_index < len(WordCountList): # Insert this data into the list at index test_index
        WordCountList.insert(test_index, [word_count, words])
        
        
    return WordCountList, test_index

WordCountList = []
curr_index = 0
for line in sys.stdin:
    word_sep = ';'
    try:
        word_count, words = unicode(line.strip()).split('\t',1)
        wordList = words.split(word_sep)
    except ValueError as e:
        continue
    word_count = int(word_count)
    WordCountList, curr_index = AddElementtoList (WordCountList, word_count, wordList, curr_index)

for WordCount in WordCountList:
    for word in WordCount[1]:
        print "%s\t%d" % (word, WordCount[0])

Overwriting T1reducer.py


In [15]:
%%bash


OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Reverse Sorted WordCount" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator='\t' \
    -D mapreduce.partition.keycomparator.options="-k1rn" \
    -files T1mapper_out.py,T1reducer.py,T1combiner.py \
    -mapper "python T1mapper_out.py" \
    -combiner "python T1combiner.py" \
    -reducer "python T1reducer.py" \
    -input ${OUT_DIR} \
    -output reverse_wordcount_sanj_result > /dev/null

hdfs dfs -cat reverse_wordcount_sanj_result/part-00000 | head -7 | tail -1

is	126420


18/02/13 17:33:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/13 17:33:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/13 17:33:09 INFO mapred.FileInputFormat: Total input files to process : 1
18/02/13 17:33:09 INFO mapreduce.JobSubmitter: number of splits:2
18/02/13 17:33:09 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1518474324860_0008
18/02/13 17:33:10 INFO impl.YarnClientImpl: Submitted application application_1518474324860_0008
18/02/13 17:33:10 INFO mapreduce.Job: The url to track the job: http://561c2dc5305b:8088/proxy/application_1518474324860_0008/
18/02/13 17:33:10 INFO mapreduce.Job: Running job: job_1518474324860_0008
18/02/13 17:33:15 INFO mapreduce.Job: Job job_1518474324860_0008 running in uber mode : false
18/02/13 17:33:15 INFO mapreduce.Job:  map 0% reduce 0%
18/02/13 17:33:31 INFO mapreduce.Job:  map 34% reduce 0%
18/02/13 17:33:37 INFO mapreduce.Job:  map 46% reduce 0%
18/02/13 17:33:43 INFO 